In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from unidecode import unidecode
import spacy
import spacy_spanish_lemmatizer
from sklearn.feature_extraction.text  import TfidfVectorizer
from sklearn.cluster import KMeans
stopwords_sp = stopwords.words('spanish')
nlp = spacy.load("es_core_news_sm")

In [2]:
Base = pd.read_excel('SUPERVISADO.xlsx',header=0)
Base.tail(1)


,OPE,nivel,req_estudio_req_experiencia,proposito,funciones
4142,112909,Asistencial,Estudio: Diploma de Bachiller y curso de técn...,Realizar labores de apoyo a los procesos del á...,1. Apoyar la gestión documental del Fondo Depa...


In [3]:
# longitud de terminos en el corpus de cada criterio
temp= []

for i in Base['req_estudio_req_experiencia']:
    temp.append(len (i.split()))

Base['reqcount']=temp

temp1= []

for i in Base['proposito']:
    temp1.append(len (i.split()))

Base['prop_clcount']=temp1

temp2= []

for i in Base['funciones']:
    temp2.append(len (i.split()))

Base['fx_clcount']=temp2
Base

,OPE,nivel,req_estudio_req_experiencia,proposito,funciones,reqcount,prop_clcount,fx_clcount
0,785,Asesor,Estudio: Título profesional en disciplina acad...,"Dirigir la formulación y ejecución de planes, ...","1. Asesorar al alcalde, a las dependencias del...",41,49,230
1,788,Asesor,Estudio: Título de formación profesional en nú...,"Asesorar a la Secretaría de Agricultura, Ganad...",1. Asesorar en la elaboración de los estudios ...,57,42,180
2,789,Asesor,Estudio: Título profesional en núcleo básico d...,Asesorar al Secretario de Hacienda en temas re...,"1. Asesorar a la Secretaría de Hacienda, en te...",54,31,216
3,790,Asesor,Estudio: Título de formación profesional en di...,"Asesorar los planes, programas y proyectos de ...",1. Asesorar a la Secretaría de Salud en el ámb...,61,29,201
4,791,Asesor,Estudio: Título de Formación Profesional en n...,Asesorar los procesos Administrativos y Finan...,1. Asesorar al Despacho del Secretario de Salu...,55,25,280
...,...,...,...,...,...,...,...,...
4138,110944,Profesional,Estudio: Título profesional en disciplina acad...,Agilizar y garantizar eficiencia en los proces...,1. Realizar el proceso completo de contratació...,42,17,260
4139,111001,Asistencial,Estudio: Titulo de bachiller en cualquier moda...,Realizar actividades de apoyo a procesos admin...,4. Atender y orientar al público personal y te...,13,25,110
4140,111006,Asistencial,Estudio: Titulo de bachiller en cualquier moda...,Realizar actividades de apoyo a los procesos a...,1. Elaborar bases de datos que se requieran po...,13,26,112
4141,112643,Tecnico,Estudio: Terminación y aprobación de tres (3) ...,Velar por el orden del flujo vehicular en las ...,1. Ejercer control y vigilancia en el cumplimi...,17,34,300


In [4]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto= unidecode(texto)
    texto = texto.split() # tokenización 
    texto = [palabra for palabra in texto if palabra not in stopwords_sp]
    texto = nlp(str(texto)) # lematización
    text=[]
    for token in texto:        
        text.append(token.lemma_)
    texto = " ".join(text)
    return texto 

In [5]:
%%time
#Limpieza y stopwords requisitos de estudio y experiencia
Base['req_cl']= Base['req_estudio_req_experiencia'].apply(lambda val: pre_procesado(val))
#Limpieza y stopwords proposito
Base['prop_cl']= Base['proposito'].apply(lambda val: pre_procesado(val))
#Limpieza y stopwords funciones 
Base['fx_cl']= Base['funciones'].apply(lambda val: pre_procesado(val))

Wall time: 8min 30s


In [6]:
Base['Contenido_cl']= Base['req_cl']+ Base['prop_cl']+ Base['fx_cl']
Base

,OPE,nivel,req_estudio_req_experiencia,proposito,funciones,reqcount,prop_clcount,fx_clcount,req_cl,prop_cl,fx_cl,Contenido_cl
0,785,Asesor,Estudio: Título profesional en disciplina acad...,"Dirigir la formulación y ejecución de planes, ...","1. Asesorar al alcalde, a las dependencias del...",41,49,230,"[ ' estudio ' , ' titulo ' , ' profesional ' ,...","[ ' dirigir ' , ' formulacion ' , ' ejecucion ...","[ ' asesorar ' , ' alcalde ' , ' dependencia '...","[ ' estudio ' , ' titulo ' , ' profesional ' ,..."
1,788,Asesor,Estudio: Título de formación profesional en nú...,"Asesorar a la Secretaría de Agricultura, Ganad...",1. Asesorar en la elaboración de los estudios ...,57,42,180,"[ ' estudio ' , ' titulo ' , ' formacion ' , '...","[ ' asesorar ' , ' secretaria ' , ' agricultur...","[ ' asesorar ' , ' elaboracion ' , ' estudio '...","[ ' estudio ' , ' titulo ' , ' formacion ' , '..."
2,789,Asesor,Estudio: Título profesional en núcleo básico d...,Asesorar al Secretario de Hacienda en temas re...,"1. Asesorar a la Secretaría de Hacienda, en te...",54,31,216,"[ ' estudio ' , ' titulo ' , ' profesional ' ,...","[ ' asesorar ' , ' secretario ' , ' hacienda '...","[ ' asesorar ' , ' secretaria ' , ' hacienda '...","[ ' estudio ' , ' titulo ' , ' profesional ' ,..."
3,790,Asesor,Estudio: Título de formación profesional en di...,"Asesorar los planes, programas y proyectos de ...",1. Asesorar a la Secretaría de Salud en el ámb...,61,29,201,"[ ' estudio ' , ' titulo ' , ' formacion ' , '...","[ ' asesorar ' , ' plan ' , ' programa ' , ' p...","[ ' asesorar ' , ' secretaria ' , ' salud ' , ...","[ ' estudio ' , ' titulo ' , ' formacion ' , '..."
4,791,Asesor,Estudio: Título de Formación Profesional en n...,Asesorar los procesos Administrativos y Finan...,1. Asesorar al Despacho del Secretario de Salu...,55,25,280,"[ ' estudio ' , ' titulo ' , ' formacion ' , '...","[ ' asesorar ' , ' proceso ' , ' administrativ...","[ ' asesorar ' , ' despacho ' , ' secretario '...","[ ' estudio ' , ' titulo ' , ' formacion ' , '..."
...,...,...,...,...,...,...,...,...,...,...,...,...
4138,110944,Profesional,Estudio: Título profesional en disciplina acad...,Agilizar y garantizar eficiencia en los proces...,1. Realizar el proceso completo de contratació...,42,17,260,"[ ' estudio ' , ' titulo ' , ' profesional ' ,...","[ ' agilizar ' , ' garantizar ' , ' eficiencia...","[ ' realizar ' , ' proceso ' , ' completo ' , ...","[ ' estudio ' , ' titulo ' , ' profesional ' ,..."
4139,111001,Asistencial,Estudio: Titulo de bachiller en cualquier moda...,Realizar actividades de apoyo a procesos admin...,4. Atender y orientar al público personal y te...,13,25,110,"[ ' estudio ' , ' titulo ' , ' bachiller ' , '...","[ ' realizar ' , ' actividad ' , ' apoyo ' , '...","[ ' atender ' , ' orientar ' , ' publico ' , '...","[ ' estudio ' , ' titulo ' , ' bachiller ' , '..."
4140,111006,Asistencial,Estudio: Titulo de bachiller en cualquier moda...,Realizar actividades de apoyo a los procesos a...,1. Elaborar bases de datos que se requieran po...,13,26,112,"[ ' estudio ' , ' titulo ' , ' bachiller ' , '...","[ ' realizar ' , ' actividad ' , ' apoyo ' , '...","[ ' elaborar ' , ' base ' , ' dato ' , ' reque...","[ ' estudio ' , ' titulo ' , ' bachiller ' , '..."
4141,112643,Tecnico,Estudio: Terminación y aprobación de tres (3) ...,Velar por el orden del flujo vehicular en las ...,1. Ejercer control y vigilancia en el cumplimi...,17,34,300,"[ ' estudio ' , ' terminacion ' , ' aprobacion...","[ ' velar ' , ' orden ' , ' flujo ' , ' vehicu...","[ ' ejercer ' , ' control ' , ' vigilancia ' ,...","[ ' estudio ' , ' terminacion ' , ' aprobacion..."


In [7]:
Asesor=Base[Base['nivel']=='Asesor']
Profesional=Base[Base['nivel']=='Profesional']
Tecnico=Base[Base['nivel']=='Tecnico']
Asistencial=Base[Base['nivel']=='Asistencial']

In [8]:
%%time
#vectorización TF-IDF
tfidf_vectorizer = TfidfVectorizer()
#nivel asesor
Asesor_con_cl_tfidf = tfidf_vectorizer.fit_transform(Asesor['Contenido_cl'])
#nivel profesional
Profesional_con_cl_tfidf = tfidf_vectorizer.fit_transform(Profesional['Contenido_cl'])
#nivel técnico
Tecnico_con_cl_tfidf = tfidf_vectorizer.fit_transform(Tecnico['Contenido_cl'])
#nivel asistencial
Asistencial_con_cl_tfidf = tfidf_vectorizer.fit_transform(Asistencial['Contenido_cl'])

Wall time: 1.09 s


In [9]:
## Training and testing bases

from sklearn.model_selection import train_test_split

Asesor_test, Asesor_train = train_test_split(Asesor_con_cl_tfidf, random_state=0,
                                  train_size=0.3)

Profesional_test, Profesional_train = train_test_split(Profesional_con_cl_tfidf, random_state=0,
                                  train_size=0.5)

Tecnico_test, Tecnico_train = train_test_split(Tecnico_con_cl_tfidf, random_state=0,
                                  train_size=0.5)

Asistencial_test, Asistencial_train = train_test_split(Asistencial_con_cl_tfidf, random_state=0,
                                  train_size=0.5)

In [10]:
### Hierchical clustering

# linkage:average, ward, ward.D2
# Distancia:euclidean tfidf, coseno lsa, coseno tfidf

